In [1]:
import numpy as np
import math
from scipy import linalg as LA
import matplotlib.pyplot as plt
from scipy.misc import derivative

In [2]:
##############################################
###                 system                 ###
##############################################

def partial_derivative(function, variable=0, point=[]):
    arguments = point[:]
    def wraps(x):
        arguments[variable] = x
        return function(*arguments)
    return derivative(wraps, point[variable], dx=np.sqrt(np.finfo(float).eps))

##############################################
b, d, q = 2, 3, 3

def lnR1(ja, jb, h):
    return 2*ja+4*h+math.log(1+(q-1)*math.exp(-2*ja-2*h))
def lnR2(ja, jb, h):
    return ja+3*h+math.log(1+math.exp(jb-ja-2*h)+(q-2)*math.exp(-ja-2*h))
def lnR3(ja, jb, h):
    return jb+math.log(2+math.exp(2*h-jb)+(q-3)*math.exp(-jb))
def lnR4(ja, jb, h):
    return 2*jb+math.log(1+math.exp(2*h-2*jb)+(q-2)*math.exp(-2*jb))

def Ja(ja, jb, h):
    jja, jjb, hh = b**(d-1)*ja, b**(d-1)*jb, b**(d-1)*h
    return lnR1(jja,jjb,hh)+lnR3(jja,jjb,hh)-2*lnR2(jja,jjb,hh)
def Jb(ja, jb, h):
    jja, jjb, hh = b**(d-1)*ja, b**(d-1)*jb, b**(d-1)*h
    return lnR4(jja,jjb,hh)-lnR3(jja,jjb,hh)
def H(ja, jb, h):
    jja, jjb, hh = b**(d-1)*ja, b**(d-1)*jb, b**(d-1)*h
    return lnR2(jja,jjb,hh)-lnR3(jja,jjb,hh)
def G(ja, jb, h):
    jja, jjb, hh = b**(d-1)*ja, b**(d-1)*jb, b**(d-1)*h
    return lnR3(jja,jjb,hh)

def T_matrix(ja, jb, h):
    
    X = partial_derivative(G,  0, [ja,jb,h])  # dG' /dJa 
    Y = partial_derivative(Ja, 0, [ja,jb,h])  # dJa'/dJa
    Z = partial_derivative(Jb, 0, [ja,jb,h])  # dJb'/dJa
    Q = partial_derivative(H,  0, [ja,jb,h])  # dH' /dJa
    
    K = partial_derivative(G,  1, [ja,jb,h])   # dG' /dJb
    L = partial_derivative(Ja, 1, [ja,jb,h])   # dJa'/dJb
    M = partial_derivative(Jb, 1, [ja,jb,h])   # dJb'/dJb
    N = partial_derivative(H,  1, [ja,jb,h])   # dH' /dJb
    
    O = partial_derivative(G,  2, [ja,jb,h])   # dG' /dH
    P = partial_derivative(Ja, 2, [ja,jb,h])   # dJa'/dH
    R = partial_derivative(Jb, 2, [ja,jb,h])   # dJb'/dH
    S = partial_derivative(H,  2, [ja,jb,h])   # dH' /dH
    
    return [[b**d, X,   K,   O],
            [0,    Y,   L,   P],
            [0,    Z,   M,   R],
            [0,    Q,   N,   S]]

In [21]:
##############################################
###            critical point              ###
##############################################

def CriticalPoint():
        
    Jc = 0
    j_initial = 0.1
    
    h = 0
    control = 0
    while control == 0:
        
        ja, jb = j_initial, j_initial
            
        Ja_test = []
        Jb_test = []
        for i in range(2):
            Ja_test.append(ja)
            Jb_test.append(jb)
            ja, jb = Ja(ja,jb,h), Jb(ja,jb,h)
        ja, jb = j_initial, j_initial
            
        if Ja_test[0] - Jb_test[0] < 0.00001 and Ja_test[1] - Jb_test[1] < 0.00001:
            pass
        else:
            print("Something is wrong!\nIt seems like H != 0\n")
            break
        
        if Ja_test[1] < Ja_test[0]:
            control = 0
        elif Ja_test[1] > Ja_test[0]:
            control = 1
            Jc = (Ja_test[0]+Ja_test[1])/2
        else:
            control = 1
            Jc = Ja_test[0]
                
        j_initial = j_initial + 0.0000001
            
    return Jc


Jc = CriticalPoint()
if Jc != 0:
    print("Jc   = ", Jc, "\n1/Jc = ", 1/Jc)

Jc   =  0.1649552287195008 
1/Jc =  6.0622509984236785


In [3]:
Jc = 0.1649551445139490341

In [ ]:
##############################################
###             just RG flows              ###
##############################################

#Jc = 0.1649552287195008
j_initial = 0.16
h_initial = 0
n = 10

#q = 3
ja, jb, h = j_initial, j_initial, h_initial
print("Ja                      Jb                      H\n")
for i in range(n):
    print(ja, "   ", jb, "   ", h)
    ja, jb, h = Ja(ja,jb,h), Jb(ja,jb,h), H(ja,jb,h)
ja, jb, h = j_initial, j_initial, h_initial
print("\n\nln[R(aa)]             ln[R(bb)]             ln[R(ab)]            ln[R(bc)]\n")
for i in range(n):
    print(lnR1(ja, jb, h), "  ", lnR4(ja, jb, h), "  ", lnR2(ja, jb, h), "  ", lnR3(ja, jb, h))
    ja, jb, h = Ja(ja,jb,h), Jb(ja,jb,h), H(ja,jb,h)

In [32]:
##############################################
###     eigenvalues and eigenvectors       ###
##############################################

T = T_matrix(Jc, Jc, 0)
w, vleft, vright = LA.eig(T, left=True)
print("T matrix:\n\n", T, "\n\n\neigenvalues:\n\n", w, "\n\n\nleft eigenvectors:\n\n", vleft)

T matrix:

 [[8, 0.0, 3.178457587957382, 1.6430848315358162], [0, 2.0350948721170425, 0.0, 0.49972210824489594], [0, 0.0, 2.0350948721170425, -0.24986106157302856], [0, 1.589228793978691, -1.589228793978691, 5.535372756421566]] 


eigenvalues:

 [8.        +0.j 5.84781605+0.j 1.72265158+0.j 2.03509487+0.j] 


left eigenvectors:

 [[ 7.90029271e-01  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 1.40324921e-01 -3.59078858e-01 -7.00371514e-01 -4.47213606e-01]
 [ 2.80649843e-01  3.59078858e-01  7.00371514e-01 -8.94427186e-01]
 [ 5.26686181e-01 -8.61466626e-01  1.37693441e-01  5.13295266e-17]]


In [33]:
2.0350948721170425/math.exp(2)

0.27542014093133804

In [9]:
[8/8        , 0.32640535/8, 3.66060968/8, 2.68070135/8]

[1.0, 0.04080066875, 0.45757621, 0.33508766875]

In [7]:
##############################################
###         RG flows and T matrix          ###
##############################################

j_initial = 0.16496
h_initial = 0
N = 19

b, d, q = 2, 3, 3

M1 = [1, 1, 0, 2]
M2 = [1, 0, 1, 0]
M3 = [1, 1/(q**2), (1-1/q)*(1/q), 2/q]
M4 = [1, 1, 1, 2]
M5 = [1, 1, 1, -2]

#Jc = 0.1649551445139490341 #CriticalPoint()
#print("Jc =", Jc, "\n")

ja, jb, h = j_initial, j_initial, h_initial

if j_initial < Jc:
    if h_initial > 0.00001:
        Mn = M4
    elif h_initial < -0.00001:
        Mn = M5
    else:
        Mn = M3

elif j_initial > Jc:
    if h_initial > 0.00001:
        Mn = M4
    elif h_initial < -0.00001:
        Mn = M5
    else:
        Mn = M2      #                                                     <---  M* for T = 0
    
ja, jb, h = j_initial, j_initial, h_initial
print("--- Ja' ------- Jb' --------- H' ---\n")
for i in range(N):
    #print(ja, jb, "    ", h)
    print("  ", round(ja, 5), "     ", round(jb, 5), "     ", round(h, 5))
    ja, jb, h = Ja(ja,jb,h), Jb(ja,jb,h), 0 #H(ja,jb,h)
print("\n\n---- T matrix, etc. ----\n")
ja, jb, h = j_initial, j_initial, h_initial
U = [[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]]
for i in range(N):
    T = T_matrix(ja,jb,h)
    U = (b**(-d))*np.dot(T, U)
    M = np.dot(Mn, U)
    print("T =", np.round(T, 4),"\n\nU =", np.round(U, 4), "\n\nM =", M, "\n\n")
    ja, jb, h = Ja(ja,jb,h), Jb(ja,jb,h), 0 #H(ja,jb,h)
    

print("\nM = Mn.TT..T =", M, "\nMn.Tn        =", np.dot(Mn, T_matrix(ja,jb,h))/8, "\nMn           =", Mn)

--- Ja' ------- Jb' --------- H' ---

   0.16496       0.16496       0
   0.16497       0.16497       0
   0.16498       0.16498       0
   0.165       0.165       0
   0.16504       0.16504       0
   0.16512       0.16512       0
   0.1653       0.1653       0
   0.16566       0.16566       0
   0.16639       0.16639       0
   0.16789       0.16789       0
   0.17097       0.17097       0
   0.17741       0.17741       0
   0.19123       0.19123       0
   0.22243       0.22243       0
   0.30056       0.30056       0
   0.53516       0.53516       0
   1.41763       1.41763       0
   4.97567       4.97567       0
   19.20954       19.20954       0


---- T matrix, etc. ----

T = [[ 8.      0.      3.1785  1.6431]
 [ 0.      2.0352  0.      0.4997]
 [ 0.      0.      2.0352 -0.2499]
 [ 0.      1.5892 -1.5892  5.5354]] 

U = [[ 1.      0.      0.3973  0.2054]
 [ 0.      0.2544  0.      0.0625]
 [ 0.      0.      0.2544 -0.0312]
 [ 0.      0.1987 -0.1987  0.6919]] 

M = [1.         0

In [19]:
##############################################
###           limit vector check           ###
##############################################

j_initial = 0.05
h_initial = 0

M1 = [1, 1, 0, 2]
M2 = [1, 0, 1, 0]
M3 = [1, 1/(q**2), (1-1/q)*(1/q), 2/q]

ja, jb, h = j_initial, j_initial, h_initial
M_results =  []

J_test = []
for i in range(2):
    J_test.append(ja)
    ja, jb, h = Ja(ja,jb,h), Jb(ja,jb,h), H(ja,jb,h)
ja, jb, h = j_initial, j_initial, h_initial


if J_test[1] < J_test[0]:
    Mn = M3
    print("choice of M* = M3 =", Mn)
    for n in range(1,3):
        ja, jb, h = j_initial, j_initial, h_initial
        U = [[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]]
        for i in range(n):
            U = (b**(-d))*np.dot(T_matrix(ja,jb,h), U)
            ja, jb, h = Ja(ja,jb,h), Jb(ja,jb,h), H(ja,jb,h)
        M = np.dot(Mn, U)
        M_results.append(M)
    M_difference = np.subtract(M_results[1], M_results[0])
    
    k = 3
    while any(i > 0.00001 for i in M_difference):
        ja, jb, h = j_initial, j_initial, h_initial
        U = [[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]]
        for i in range(k):
            U = (b**(-d))*np.dot(T_matrix(ja,jb,h), U)
            ja, jb, h = Ja(ja,jb,h), Jb(ja,jb,h), H(ja,jb,h)
        M = np.dot(Mn, U)
        M_results.append(M)
        M_difference = np.subtract(M_results[k-1], M_results[k-2])
        k = k + 1
        
        
elif J_test[1] > J_test[0]:
    Mn = M1         #                                       <---  M* for T = 0
    print("choice of M* = M1 =", Mn)
    for n in range(1,3):
        ja, jb, h = j_initial, j_initial, h_initial
        U = [[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]]
        for i in range(n):
            U = (b**(-d))*np.dot(T_matrix(ja,jb,h), U)
            ja, jb, h = Ja(ja,jb,h), Jb(ja,jb,h), H(ja,jb,h)
        M = np.dot(Mn, U)
        M_results.append(M)
    M_difference = np.subtract(M_results[1], M_results[0])
    
    k = 3
    while any(i > 0.00001 for i in M_difference):
        ja, jb, h = j_initial, j_initial, h_initial
        U = [[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]]
        for i in range(k):
            U = (b**(-d))*np.dot(T_matrix(ja,jb,h), U)
            ja, jb, h = Ja(ja,jb,h), Jb(ja,jb,h), H(ja,jb,h)
        M = np.dot(Mn, U)
        M_results.append(M)
        M_difference = np.subtract(M_results[k-1], M_results[k-2])
        k = k + 1

print("\n\nM vectors: ", M_results, "\n\n\nn = ", k-1, "\n\nM difference: ", M_difference)

choice of M* = M3 = [1, 0.1111111111111111, 0.22222222222222224, 0.6666666666666666]


M vectors:  [array([1.        , 0.12630809, 0.25261617, 0.66666667]), array([1.        , 0.12661433, 0.25322865, 0.66666667]), array([1.        , 0.12661478, 0.25322956, 0.66666667])] 


n =  3 

M difference:  [ 0.00000000e+00  4.51695297e-07  9.03456507e-07 -6.12045969e-11]


In [28]:
j_initial = 0.1
h_initial = 0
n = 10

b, d, q = 2, 3, 3

M3 = [1, 1/(q**2), (1-1/q)*(1/q), 2/q]

#Jc = 0.1649551445139490341 #CriticalPoint()
#print("Jc =", Jc, "\n")
while j_initial < Jc:
    
    ja, jb, h = j_initial, j_initial, h_initial
    
    Mn = M3
    
    U = [[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]]
    for i in range(n):
        U = (b**(-d))*np.dot(T_matrix(ja,jb,h), U)
        ja, jb, h = Ja(ja,jb,h), Jb(ja,jb,h), 0
    M = np.dot(Mn, U)
        
    print(round(1/j_initial, 3), "  ", M)
    
    j_initial = j_initial + 0.001

10.0    [1.         0.1445531  0.2891062  0.66666667]
9.901    [1.         0.14495167 0.28990333 0.66666667]
9.804    [1.         0.14535224 0.29070448 0.66666667]
9.709    [1.         0.14575485 0.29150971 0.66666667]
9.615    [1.         0.14615954 0.29231908 0.66666667]
9.524    [1.         0.14656634 0.29313268 0.66666667]
9.434    [1.         0.14697528 0.29395056 0.66666667]
9.346    [1.         0.14738641 0.29477282 0.66666667]
9.259    [1.         0.14779976 0.29559951 0.66666667]
9.174    [1.         0.14821537 0.29643073 0.66666667]
9.091    [1.         0.14863328 0.29726655 0.66666667]
9.009    [1.         0.14905353 0.29810706 0.66666667]
8.929    [1.         0.14947617 0.29895235 0.66666667]
8.85    [1.         0.14990125 0.29980249 0.66666667]
8.772    [1.         0.1503288  0.3006576  0.66666667]
8.696    [1.         0.15075887 0.30151775 0.66666667]
8.621    [1.         0.15119152 0.30238305 0.66666667]
8.547    [1.         0.1516268  0.30325359 0.66666667]
8.475    [1.

In [4]:
j_initial = 20
h_initial = 0
n = 10

b, d, q = 2, 3, 3

M1 = [1, 1, 0, 2]
M2 = [1, 0, 1, 0]


#Jc = 0.1649551445139490341 #CriticalPoint()
print("Jc =", Jc, "\n")
while j_initial > Jc:
    
    ja, jb, h = j_initial, j_initial, h_initial
    
    Mn = M1      #                                                   <---  M* for T = 0
    
    U = [[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]]
    for i in range(n):
        U = (b**(-d))*np.dot(T_matrix(ja,jb,h), U)
        ja, jb, h = Ja(ja,jb,h), Jb(ja,jb,h), 0
    M = np.dot(Mn, U)
        
    print(round(j_initial, 3), "   ", round(1/j_initial, 3), "  ", M)
    
    j_initial = j_initial - 0.01

Jc = 0.16495514451394902 

20     0.05    [1. 1. 0. 2.]
19.99     0.05    [1. 1. 0. 2.]
19.98     0.05    [1. 1. 0. 2.]
19.97     0.05    [1. 1. 0. 2.]
19.96     0.05    [1. 1. 0. 2.]
19.95     0.05    [1. 1. 0. 2.]
19.94     0.05    [1. 1. 0. 2.]
19.93     0.05    [1. 1. 0. 2.]
19.92     0.05    [1. 1. 0. 2.]
19.91     0.05    [1. 1. 0. 2.]
19.9     0.05    [1. 1. 0. 2.]
19.89     0.05    [1. 1. 0. 2.]
19.88     0.05    [1. 1. 0. 2.]
19.87     0.05    [1. 1. 0. 2.]
19.86     0.05    [1. 1. 0. 2.]
19.85     0.05    [1. 1. 0. 2.]
19.84     0.05    [1. 1. 0. 2.]
19.83     0.05    [1. 1. 0. 2.]
19.82     0.05    [1. 1. 0. 2.]
19.81     0.05    [1. 1. 0. 2.]
19.8     0.051    [1. 1. 0. 2.]
19.79     0.051    [1. 1. 0. 2.]
19.78     0.051    [1. 1. 0. 2.]
19.77     0.051    [1. 1. 0. 2.]
19.76     0.051    [1. 1. 0. 2.]
19.75     0.051    [1. 1. 0. 2.]
19.74     0.051    [1. 1. 0. 2.]
19.73     0.051    [1. 1. 0. 2.]
19.72     0.051    [1. 1. 0. 2.]
19.71     0.051    [1. 1. 0. 2.]
19.7    

17.4     0.057    [1. 1. 0. 2.]
17.39     0.058    [1. 1. 0. 2.]
17.38     0.058    [1. 1. 0. 2.]
17.37     0.058    [1. 1. 0. 2.]
17.36     0.058    [1. 1. 0. 2.]
17.35     0.058    [1. 1. 0. 2.]
17.34     0.058    [1. 1. 0. 2.]
17.33     0.058    [1. 1. 0. 2.]
17.32     0.058    [1. 1. 0. 2.]
17.31     0.058    [1. 1. 0. 2.]
17.3     0.058    [1. 1. 0. 2.]
17.29     0.058    [1. 1. 0. 2.]
17.28     0.058    [1. 1. 0. 2.]
17.27     0.058    [1. 1. 0. 2.]
17.26     0.058    [1. 1. 0. 2.]
17.25     0.058    [1. 1. 0. 2.]
17.24     0.058    [1. 1. 0. 2.]
17.23     0.058    [1. 1. 0. 2.]
17.22     0.058    [1. 1. 0. 2.]
17.21     0.058    [1. 1. 0. 2.]
17.2     0.058    [1. 1. 0. 2.]
17.19     0.058    [1. 1. 0. 2.]
17.18     0.058    [1. 1. 0. 2.]
17.17     0.058    [1. 1. 0. 2.]
17.16     0.058    [1. 1. 0. 2.]
17.15     0.058    [1. 1. 0. 2.]
17.14     0.058    [1. 1. 0. 2.]
17.13     0.058    [1. 1. 0. 2.]
17.12     0.058    [1. 1. 0. 2.]
17.11     0.058    [1. 1. 0. 2.]
17.1     0.05

14.9     0.067    [1. 1. 0. 2.]
14.89     0.067    [1. 1. 0. 2.]
14.88     0.067    [1. 1. 0. 2.]
14.87     0.067    [1. 1. 0. 2.]
14.86     0.067    [1. 1. 0. 2.]
14.85     0.067    [1. 1. 0. 2.]
14.84     0.067    [1. 1. 0. 2.]
14.83     0.067    [1. 1. 0. 2.]
14.82     0.067    [1. 1. 0. 2.]
14.81     0.068    [1. 1. 0. 2.]
14.8     0.068    [1. 1. 0. 2.]
14.79     0.068    [1. 1. 0. 2.]
14.78     0.068    [1. 1. 0. 2.]
14.77     0.068    [1. 1. 0. 2.]
14.76     0.068    [1. 1. 0. 2.]
14.75     0.068    [1. 1. 0. 2.]
14.74     0.068    [1. 1. 0. 2.]
14.73     0.068    [1. 1. 0. 2.]
14.72     0.068    [1. 1. 0. 2.]
14.71     0.068    [1. 1. 0. 2.]
14.7     0.068    [1. 1. 0. 2.]
14.69     0.068    [1. 1. 0. 2.]
14.68     0.068    [1. 1. 0. 2.]
14.67     0.068    [1. 1. 0. 2.]
14.66     0.068    [1. 1. 0. 2.]
14.65     0.068    [1. 1. 0. 2.]
14.64     0.068    [1. 1. 0. 2.]
14.63     0.068    [1. 1. 0. 2.]
14.62     0.068    [1. 1. 0. 2.]
14.61     0.068    [1. 1. 0. 2.]
14.6     0.06

12.36     0.081    [1. 1. 0. 2.]
12.35     0.081    [1. 1. 0. 2.]
12.34     0.081    [1. 1. 0. 2.]
12.33     0.081    [1. 1. 0. 2.]
12.32     0.081    [1. 1. 0. 2.]
12.31     0.081    [1. 1. 0. 2.]
12.3     0.081    [1. 1. 0. 2.]
12.29     0.081    [1. 1. 0. 2.]
12.28     0.081    [1. 1. 0. 2.]
12.27     0.081    [1. 1. 0. 2.]
12.26     0.082    [1. 1. 0. 2.]
12.25     0.082    [1. 1. 0. 2.]
12.24     0.082    [1. 1. 0. 2.]
12.23     0.082    [1. 1. 0. 2.]
12.22     0.082    [1. 1. 0. 2.]
12.21     0.082    [1. 1. 0. 2.]
12.2     0.082    [1. 1. 0. 2.]
12.19     0.082    [1. 1. 0. 2.]
12.18     0.082    [1. 1. 0. 2.]
12.17     0.082    [1. 1. 0. 2.]
12.16     0.082    [1. 1. 0. 2.]
12.15     0.082    [1. 1. 0. 2.]
12.14     0.082    [1. 1. 0. 2.]
12.13     0.082    [1. 1. 0. 2.]
12.12     0.083    [1. 1. 0. 2.]
12.11     0.083    [1. 1. 0. 2.]
12.1     0.083    [1. 1. 0. 2.]
12.09     0.083    [1. 1. 0. 2.]
12.08     0.083    [1. 1. 0. 2.]
12.07     0.083    [1. 1. 0. 2.]
12.06     0.0

9.82     0.102    [1. 1. 0. 2.]
9.81     0.102    [1. 1. 0. 2.]
9.8     0.102    [1. 1. 0. 2.]
9.79     0.102    [1. 1. 0. 2.]
9.78     0.102    [1. 1. 0. 2.]
9.77     0.102    [1. 1. 0. 2.]
9.76     0.102    [1. 1. 0. 2.]
9.75     0.103    [1. 1. 0. 2.]
9.74     0.103    [1. 1. 0. 2.]
9.73     0.103    [1. 1. 0. 2.]
9.72     0.103    [1. 1. 0. 2.]
9.71     0.103    [1. 1. 0. 2.]
9.7     0.103    [1. 1. 0. 2.]
9.69     0.103    [1. 1. 0. 2.]
9.68     0.103    [1. 1. 0. 2.]
9.67     0.103    [1. 1. 0. 2.]
9.66     0.104    [1. 1. 0. 2.]
9.65     0.104    [1. 1. 0. 2.]
9.64     0.104    [1. 1. 0. 2.]
9.63     0.104    [1. 1. 0. 2.]
9.62     0.104    [1. 1. 0. 2.]
9.61     0.104    [1. 1. 0. 2.]
9.6     0.104    [1. 1. 0. 2.]
9.59     0.104    [1. 1. 0. 2.]
9.58     0.104    [1. 1. 0. 2.]
9.57     0.104    [1. 1. 0. 2.]
9.56     0.105    [1. 1. 0. 2.]
9.55     0.105    [1. 1. 0. 2.]
9.54     0.105    [1. 1. 0. 2.]
9.53     0.105    [1. 1. 0. 2.]
9.52     0.105    [1. 1. 0. 2.]
9.51     0.

7.27     0.138    [1.         1.         0.         2.00000001]
7.26     0.138    [1. 1. 0. 2.]
7.25     0.138    [1. 1. 0. 2.]
7.24     0.138    [1. 1. 0. 2.]
7.23     0.138    [1. 1. 0. 2.]
7.22     0.139    [1. 1. 0. 2.]
7.21     0.139    [1. 1. 0. 2.]
7.2     0.139    [1. 1. 0. 2.]
7.19     0.139    [1. 1. 0. 2.]
7.18     0.139    [1. 1. 0. 2.]
7.17     0.139    [1. 1. 0. 2.]
7.16     0.14    [1. 1. 0. 2.]
7.15     0.14    [1. 1. 0. 2.]
7.14     0.14    [1. 1. 0. 2.]
7.13     0.14    [1. 1. 0. 2.]
7.12     0.14    [1. 1. 0. 2.]
7.11     0.141    [1. 1. 0. 2.]
7.1     0.141    [1. 1. 0. 2.]
7.09     0.141    [1. 1. 0. 2.]
7.08     0.141    [1. 1. 0. 2.]
7.07     0.141    [1. 1. 0. 2.]
7.06     0.142    [1. 1. 0. 2.]
7.05     0.142    [1. 1. 0. 2.]
7.04     0.142    [1. 1. 0. 2.]
7.03     0.142    [1. 1. 0. 2.]
7.02     0.142    [1. 1. 0. 2.]
7.01     0.143    [1. 1. 0. 2.]
7.0     0.143    [1. 1. 0. 2.]
6.99     0.143    [1. 1. 0. 2.]
6.98     0.143    [1. 1. 0. 2.]
6.97     0.143  

4.83     0.207    [1. 1. 0. 2.]
4.82     0.207    [1. 1. 0. 2.]
4.81     0.208    [1. 1. 0. 2.]
4.8     0.208    [1. 1. 0. 2.]
4.79     0.209    [1. 1. 0. 2.]
4.78     0.209    [1. 1. 0. 2.]
4.77     0.21    [1. 1. 0. 2.]
4.76     0.21    [ 1.00000000e+00  1.00000000e+00 -1.49011612e-08  2.00000001e+00]
4.75     0.211    [1. 1. 0. 2.]
4.74     0.211    [1. 1. 0. 2.]
4.73     0.211    [1. 1. 0. 2.]
4.72     0.212    [1. 1. 0. 2.]
4.71     0.212    [1. 1. 0. 2.]
4.7     0.213    [1. 1. 0. 2.]
4.69     0.213    [1. 1. 0. 2.]
4.68     0.214    [1.         1.         0.         1.99999999]
4.67     0.214    [1. 1. 0. 2.]
4.66     0.215    [1.         1.         0.         1.99999999]
4.65     0.215    [1. 1. 0. 2.]
4.64     0.216    [1.         1.         0.         1.99999999]
4.63     0.216    [1. 1. 0. 2.]
4.62     0.216    [1. 1. 0. 2.]
4.61     0.217    [1. 1. 0. 2.]
4.6     0.217    [1. 1. 0. 2.]
4.59     0.218    [1. 1. 0. 2.]
4.58     0.218    [ 1.00000000e+00  1.00000000e+00 -1.490

3.47     0.288    [1.00000000e+00 1.00000000e+00 2.23517418e-08 1.99999998e+00]
3.46     0.289    [1.00000000e+00 1.00000000e+00 7.45058060e-09 1.99999999e+00]
3.45     0.29    [ 1.00000000e+00  1.00000000e+00 -1.49011612e-08  2.00000000e+00]
3.44     0.291    [1.00000000e+00 1.00000000e+00 2.23517418e-08 2.00000001e+00]
3.43     0.292    [1.         1.         0.         2.00000001]
3.42     0.292    [1.00000000e+00 1.00000000e+00 1.49011612e-08 2.00000000e+00]
3.41     0.293    [1. 1. 0. 2.]
3.4     0.294    [ 1.00000000e+00  1.00000000e+00 -2.23517418e-08  2.00000001e+00]
3.39     0.295    [1.         1.         0.         1.99999999]
3.38     0.296    [ 1.00000000e+00  1.00000000e+00 -2.23517418e-08  2.00000001e+00]
3.37     0.297    [ 1.00000000e+00  1.00000000e+00 -1.49011612e-08  2.00000001e+00]
3.36     0.298    [1.00000000e+00 1.00000000e+00 7.45058060e-09 1.99999999e+00]
3.35     0.299    [1.00000000e+00 1.00000000e+00 2.23517418e-08 1.99999999e+00]
3.34     0.299    [1.00000

2.33     0.429    [ 1.00000000e+00  9.99999985e-01 -7.45058060e-09  1.99999998e+00]
2.32     0.431    [ 1.00000000e+00  9.99999985e-01 -1.49011612e-08  1.99999999e+00]
2.31     0.433    [ 1.00000000e+00  9.99999985e-01 -7.45058060e-09  1.99999998e+00]
2.3     0.435    [1.00000000e+00 9.99999970e-01 1.49011612e-08 1.99999998e+00]
2.29     0.437    [ 1.00000000e+00  9.99999970e-01 -7.45058060e-09  1.99999996e+00]
2.28     0.439    [1.         0.99999997 0.         1.99999998]
2.27     0.441    [1.00000000e+00 9.99999970e-01 7.45058060e-09 1.99999998e+00]
2.26     0.442    [ 1.00000000e+00  9.99999970e-01 -7.45058060e-09  1.99999998e+00]
2.25     0.444    [ 1.00000000e+00  9.99999970e-01 -7.45058060e-09  1.99999998e+00]
2.24     0.446    [1.         0.99999997 0.         1.99999998]
2.23     0.448    [1.00000000e+00 9.99999970e-01 7.45058060e-09 1.99999996e+00]
2.22     0.45    [1.         0.99999997 0.         1.99999996]
2.21     0.452    [1.         0.99999996 0.         1.99999996]
2.

1.11     0.901    [1.00000000e+00 9.99721792e-01 1.12372538e-08 1.99972180e+00]
1.1     0.909    [1.00000000e+00 9.99698630e-01 1.91722171e-09 1.99969864e+00]
1.09     0.917    [1.00000000e+00 9.99673529e-01 9.36505484e-09 1.99967352e+00]
1.08     0.926    [ 1.00000000e+00  9.99646353e-01 -1.30921346e-08  1.99964636e+00]
1.07     0.935    [1.00000000e+00 9.99616904e-01 1.84984161e-09 1.99961690e+00]
1.06     0.943    [ 1.00000000e+00  9.99585003e-01 -2.22044605e-16  1.99958501e+00]
1.05     0.952    [ 1.00000000e+00  9.99550469e-01 -3.72529052e-09  1.99955047e+00]
1.04     0.962    [1.00000000e+00 9.99513043e-01 1.87344018e-09 1.99951303e+00]
1.03     0.971    [ 1.00000000e+00  9.99472512e-01 -1.90373617e-09  1.99947251e+00]
1.02     0.98    [ 1.00000000e+00  9.99428609e-01 -1.82005966e-09  1.99942861e+00]
1.01     0.99    [ 1.00000000e+00  9.99381039e-01 -1.50172942e-08  1.99938103e+00]
1.0     1.0    [ 1.00000000e+00  9.99329528e-01 -1.30166178e-08  1.99932955e+00]
0.99     1.01    [

In [34]:
##############################################
###                 system                 ###
##############################################

def partial_derivative(function, variable=0, point=[]):
    arguments = point[:]
    def wraps(x):
        arguments[variable] = x
        return function(*arguments)
    return derivative(wraps, point[variable], dx=np.sqrt(np.finfo(float).eps))

##############################################
b, d, q = 2, 3, 3

def lnR1(ja, jb, h):
    return 2*ja+4*h+math.log(1+(q-1)*math.exp(-2*ja-2*h))
def lnR2(ja, jb, h):
    return ja+3*h+math.log(1+math.exp(jb-ja-2*h)+(q-2)*math.exp(-ja-2*h))
def lnR3(ja, jb, h):
    return jb+2*h+math.log(2*math.exp(-2*h)+math.exp(-jb)+(q-3)*math.exp(-jb-2*h))
def lnR4(ja, jb, h):
    return 2*jb+2*h+math.log(math.exp(-2*h)+math.exp(-2*jb)+(q-2)*math.exp(-2*jb-2*h))

def Ja(ja, jb, h):
    jja, jjb, hh = b**(d-1)*ja, b**(d-1)*jb, b**(d-1)*h
    return lnR1(jja,jjb,hh)+lnR3(jja,jjb,hh)-2*lnR2(jja,jjb,hh)
def Jb(ja, jb, h):
    jja, jjb, hh = b**(d-1)*ja, b**(d-1)*jb, b**(d-1)*h
    return lnR4(jja,jjb,hh)-lnR3(jja,jjb,hh)
def H(ja, jb, h):
    jja, jjb, hh = b**(d-1)*ja, b**(d-1)*jb, b**(d-1)*h
    return lnR2(jja,jjb,hh)-lnR3(jja,jjb,hh)
def G(ja, jb, h):
    jja, jjb, hh = b**(d-1)*ja, b**(d-1)*jb, b**(d-1)*h
    return lnR3(jja,jjb,hh)

def T_matrix(ja, jb, h):
    
    X = partial_derivative(G,  0, [ja,jb,h])  # dG' /dJa 
    Y = partial_derivative(Ja, 0, [ja,jb,h])  # dJa'/dJa
    Z = partial_derivative(Jb, 0, [ja,jb,h])  # dJb'/dJa
    Q = partial_derivative(H,  0, [ja,jb,h])  # dH' /dJa
    
    K = partial_derivative(G,  1, [ja,jb,h])   # dG' /dJb
    L = partial_derivative(Ja, 1, [ja,jb,h])   # dJa'/dJb
    M = partial_derivative(Jb, 1, [ja,jb,h])   # dJb'/dJb
    N = partial_derivative(H,  1, [ja,jb,h])   # dH' /dJb
    
    O = partial_derivative(G,  2, [ja,jb,h])   # dG' /dH
    P = partial_derivative(Ja, 2, [ja,jb,h])   # dJa'/dH
    R = partial_derivative(Jb, 2, [ja,jb,h])   # dJb'/dH
    S = partial_derivative(H,  2, [ja,jb,h])   # dH' /dH
    
    return [[1,    b**(-d)*X,   b**(-d)*K,   b**(-d)*O],
            [0,    b**(-d)*Y,   b**(-d)*L,   b**(-d)*P],
            [0,    b**(-d)*Z,   b**(-d)*M,   b**(-d)*R],
            [0,    b**(-d)*Q,   b**(-d)*N,   b**(-d)*S]]

j_initial = 0.16496
h_initial = 0
N = 18
n = 1
b, d, q = 2, 3, 3

M1 = [1, 1, 0, 2]
M2 = [1, 0, 1, 0]
M3 = [1, 1/(q**2), (1-1/q)*(1/q), 2/q]

#Jc = 0.1649551445139490341 #CriticalPoint()
#print("Jc =", Jc, "\n")

ja, jb, h = j_initial, j_initial, h_initial

if j_initial < Jc:
        Mn = M3

elif j_initial > Jc:
        Mn = M1      #                                       <---  M* for T = 0
    
ja, jb, h = j_initial, j_initial, h_initial
print("n = 0     J =", ja, "\n")
U = [[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]]
for i in range(N):
    ja, jb, h = Ja(ja,jb,h), Jb(ja,jb,h), 0
    T = T_matrix(ja,jb,h)
    U = np.dot(T, U)
    print("n =", n, "    J = ", round(ja, 4), "\n", np.round(T,3), "\n\n", np.round(U,3), "\n\n")
    n = n +1

n = 0     J = 0.16496 

n = 1     J =  0.165 
 [[ 1.     0.     0.397  0.205]
 [ 0.     0.254  0.     0.062]
 [ 0.     0.     0.254 -0.031]
 [ 0.     0.199 -0.199  0.692]] 

 [[ 1.     0.     0.397  0.205]
 [ 0.     0.254  0.     0.062]
 [ 0.     0.     0.254 -0.031]
 [ 0.     0.199 -0.199  0.692]] 


n = 2     J =  0.165 
 [[ 1.     0.     0.397  0.205]
 [ 0.     0.254  0.     0.062]
 [ 0.     0.     0.254 -0.031]
 [ 0.     0.199 -0.199  0.692]] 

 [[ 1.     0.041  0.458  0.335]
 [ 0.     0.077 -0.012  0.059]
 [ 0.    -0.006  0.071 -0.03 ]
 [ 0.     0.188 -0.188  0.497]] 


n = 3     J =  0.165 
 [[ 1.     0.     0.397  0.205]
 [ 0.     0.254  0.     0.062]
 [ 0.     0.     0.254 -0.031]
 [ 0.     0.199 -0.199  0.692]] 

 [[ 1.     0.077  0.447  0.425]
 [ 0.     0.031 -0.015  0.046]
 [ 0.    -0.007  0.024 -0.023]
 [ 0.     0.147 -0.147  0.362]] 


n = 4     J =  0.165 
 [[ 1.     0.     0.397  0.205]
 [ 0.     0.255  0.     0.063]
 [ 0.     0.     0.255 -0.031]
 [ 0.     0.199 -0.199 

In [35]:
np.dot(M1, U)

array([1.        , 0.18760658, 0.34527609, 0.69060689])